# truck-n-trailer

<a href="https://colab.research.google.com/github/ericbreh/truck-n-trailer/blob/main/truck-n-trailer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !pip install polytope
  !pip install -q pyomo
  !apt-get install -y -qq glpk-utils
  !apt-get install -y -qq coinor-cbc
  !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
  !unzip -o -q ipopt-linux64

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
# Increase animation embed limit to avoid size warnings (set to 150 MB)
matplotlib.rcParams['animation.embed_limit'] = 150
import pyomo.environ as pyo
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

### Core Dynamics

- states: `[x, y, θ_truck, θ_trailer]`
- controls: `[v, φ]` (velocity and steering angle)

In [ ]:
# --- Code 2: Dynamics ---
def truck_trailer_dynamics(q, u, L, d):
    # q = [x, y, theta_t, theta_l, v, phi]
    # u = [a, phi_dot]
    x, y, theta_t, theta_l, v, phi = q
    a, phi_dot = u

    x_dot = v * np.cos(theta_t)
    y_dot = v * np.sin(theta_t)
    theta_t_dot = (v / L) * np.tan(phi)
    theta_l_dot = (v / d) * np.sin(theta_t - theta_l)
    v_dot = a
    phi_dot = phi_dot

    return np.array([x_dot, y_dot, theta_t_dot, theta_l_dot, v_dot, phi_dot])


### Pyomo Model

In [ ]:
# --- Code 3: Pyomo Model ---
def build_pyomo_model(params):
    N = params["N"]
    dt = params["dt"]
    L = params["L"]
    d = params["d"]
    target = params["target"]

    # Control bounds
    a_min = params.get("a_min", -2.0)
    a_max = params.get("a_max", 2.0)
    phi_dot_min = params.get("phi_dot_min", -1.0)
    phi_dot_max = params.get("phi_dot_max", 1.0)

    # State bounds
    v_min = params.get("v_min", 0.0)
    v_max = params.get("v_max", 3.0)
    phi_min = params.get("phi_min", -1.5)
    phi_max = params.get("phi_max", 1.5)

    w_control = params.get("w_control", 0.01)
    max_jackknife = params.get("max_jackknife_angle", np.pi/2)

    model = pyo.ConcreteModel()

    # Sets
    model.T = pyo.RangeSet(0, N)
    model.K = pyo.RangeSet(0, N - 1)
    model.S = pyo.RangeSet(0, 5)  # x, y, theta_t, theta_l, v, phi

    # State variables
    model.x = pyo.Var(model.S, model.T, domain=pyo.Reals)  

    # Control variables
    model.a = pyo.Var(model.K, bounds=(a_min, a_max))
    model.phi_dot = pyo.Var(model.K, bounds=(phi_dot_min, phi_dot_max))

    # Dynamics
    def dyn_rule(model, s, k):
        if s == 0:  # x
            return model.x[0, k+1] == model.x[0, k] + dt * (model.x[4, k] * pyo.cos(model.x[2, k]))
        if s == 1:  # y
            return model.x[1, k+1] == model.x[1, k] + dt * (model.x[4, k] * pyo.sin(model.x[2, k]))
        if s == 2:  # theta_t
            return model.x[2, k+1] == model.x[2, k] + dt * ((model.x[4, k] / L) * pyo.tan(model.x[5, k]))
        if s == 3:  # theta_l
            return model.x[3, k+1] == model.x[3, k] + dt * ((model.x[4, k] / d) * pyo.sin(model.x[2, k] - model.x[3, k]))
        if s == 4:  # v
            return model.x[4, k+1] == model.x[4, k] + dt * model.a[k]
        if s == 5:  # phi
            return model.x[5, k+1] == model.x[5, k] + dt * model.phi_dot[k]

    model.dyn = pyo.Constraint(model.S, model.K, rule=dyn_rule)

    # Jackknife constraint
    def jackknife_rule(model, t):
        return pyo.inequality(-max_jackknife, model.x[2, t] - model.x[3, t], max_jackknife)
    model.jackknife_cons = pyo.Constraint(model.T, rule=jackknife_rule)

    # Velocity & steering angle constraints
    model.vel_cons = pyo.Constraint(model.T, rule=lambda m, t: pyo.inequality(v_min, m.x[4, t], v_max))
    model.phi_cons = pyo.Constraint(model.T, rule=lambda m, t: pyo.inequality(phi_min, m.x[5, t], phi_max))

    # Objective
    def obj_rule(model):
        final_err = sum((model.x[i, N] - target[i])**2 for i in range(4))
        control_effort = sum(model.a[k]**2 + model.phi_dot[k]**2 for k in model.K)
        return final_err + w_control * control_effort
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    return model


MPC

In [ ]:
# --- Code 4: Helpers ---
def set_initial_guess(model, u_guess):
    N = u_guess.shape[1]
    for k in range(N):
        model.a[k].set_value(float(u_guess[0, k]))
        model.phi_dot[k].set_value(float(u_guess[1, k]))

def extract_solution(model, params):
    N = params["N"]
    x_opt = np.zeros((6, N+1))
    u_opt = np.zeros((2, N))
    for i in range(6):
        for t in range(N+1):
            x_opt[i, t] = pyo.value(model.x[i, t])
    for k in range(N):
        u_opt[0, k] = pyo.value(model.a[k])
        u_opt[1, k] = pyo.value(model.phi_dot[k])
    return x_opt, u_opt

def run_mpc_loop(params):
    MAX_STEPS = params.get("max_steps", 150)
    TARGET_TOL = params.get("target_tol", 0.5)

    model = build_pyomo_model(params)
    solver = pyo.SolverFactory("ipopt")
    solver.options["max_iter"] = params.get("solver_max_iter", 500)
    solver.options["tol"] = params.get("solver_tol", 1e-6)
    solver.options["print_level"] = params.get("solver_print_level", 0)

    q_current = params["q0"].copy()
    N = params["N"]

    # initial guess for controls
    u_guess = np.zeros((2, N))
    u_guess[0, :] = params.get("a_init", 0.0)
    u_guess[1, :] = params.get("phi_dot_init", 0.0)

    # Fix initial state
    for i in range(6):
        model.x[i, 0].fix(float(q_current[i]))

    q_history = [q_current.copy()]
    u_history = []
    step = 0

    print("Starting MPC simulation...")

    while np.linalg.norm(q_current[:2] - params["target"][:2]) > TARGET_TOL and step < MAX_STEPS:
        for i in range(6):
            model.x[i, 0].fix(float(q_current[i]))

        set_initial_guess(model, u_guess)
        result = solver.solve(model, tee=False)
        term = result.solver.termination_condition
        if term not in (pyo.TerminationCondition.optimal, pyo.TerminationCondition.locallyOptimal):
            print(f"Solver warning: {term} at step {step}")
            break

        x_opt, u_opt = extract_solution(model, params)
        u_star = u_opt[:, 0]
        q_dot = truck_trailer_dynamics(q_current, u_star, params["L"], params["d"])
        q_next = q_current + params["dt"] * q_dot

        q_history.append(q_next.copy())
        u_history.append(u_star.copy())

        q_current = q_next.copy()
        step += 1

        u_guess = np.hstack([u_opt[:, 1:], u_opt[:, -1:]])

    print(f"MPC finished in {step} steps. Final error: {np.linalg.norm(q_current[:2] - params['target'][:2]):.3f} m")
    return np.array(q_history), np.array(u_history)


### Simulation

In [ ]:
# --- Code 5: Plots & Animation ---
def plot_trajectory(q_hist, u_hist, params):
    q_hist = np.atleast_2d(q_hist)
    if q_hist.shape[0] == 6:
        q_hist = q_hist.T

    fig, axes = plt.subplots(3, 2, figsize=(14, 12))

    # States
    state_labels = ["x (m)", "y (m)", "theta_t (rad)", "theta_l (rad)", "v (m/s)", "phi (rad)"]
    for i, ax in enumerate(axes.flatten()):
        if i < 6:
            ax.plot(np.arange(q_hist.shape[0])*params["dt"], q_hist[:, i], "-o", markersize=3)
            ax.set_title(state_labels[i])
            ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Controls
    if len(u_hist) > 0:
        t = np.arange(len(u_hist))*params["dt"]
        plt.figure(figsize=(10,4))
        plt.step(t, u_hist[:, 0], where="post", label="a (m/s²)")
        plt.step(t, u_hist[:, 1], where="post", label="phi_dot (rad/s)")
        plt.xlabel("Time (s)")
        plt.ylabel("Control")
        plt.title("Applied Controls")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

def draw_truck_trailer_2d(ax, x, y, theta_t, theta_l, L, d, alpha=1.0, patches_list=None):
    """
    Draw a 2D representation of truck and trailer with proper shapes.
    x, y: hitch position (rear axle of truck)
    theta_t: truck heading angle
    theta_l: trailer heading angle
    L: truck wheelbase
    d: trailer length
    patches_list: list to append patches to (for cleanup in animation)
    """
    from matplotlib.patches import Rectangle, Circle, Polygon, FancyArrowPatch
    
    if patches_list is None:
        patches_list = []
    
    # Truck: from rear axle (hitch) to front axle
    truck_rear = np.array([x, y])
    truck_front = np.array([x + L*np.cos(theta_t), y + L*np.sin(theta_t)])
    
    # Trailer: from hitch to rear axle
    trailer_rear = np.array([x - d*np.cos(theta_l), y - d*np.sin(theta_l)])
    trailer_hitch = np.array([x, y])
    
    # Draw truck body (rectangle using Polygon for rotation)
    truck_center = (truck_rear + truck_front) / 2
    truck_width = 0.8
    truck_length = L
    # Create rectangle corners in local coordinates
    corners_local = np.array([
        [-truck_length/2, -truck_width/2],
        [truck_length/2, -truck_width/2],
        [truck_length/2, truck_width/2],
        [-truck_length/2, truck_width/2]
    ])
    # Rotate and translate
    rot_matrix = np.array([[np.cos(theta_t), -np.sin(theta_t)],
                           [np.sin(theta_t), np.cos(theta_t)]])
    corners_global = (rot_matrix @ corners_local.T).T + truck_center
    truck_poly = Polygon(corners_global, facecolor='lightblue', edgecolor='blue', 
                         linewidth=2, alpha=alpha, zorder=3)
    ax.add_patch(truck_poly)
    patches_list.append(truck_poly)
    
    # Draw trailer body (rectangle)
    trailer_center = (trailer_hitch + trailer_rear) / 2
    trailer_width = 0.9
    trailer_length = d
    corners_local = np.array([
        [-trailer_length/2, -trailer_width/2],
        [trailer_length/2, -trailer_width/2],
        [trailer_length/2, trailer_width/2],
        [-trailer_length/2, trailer_width/2]
    ])
    rot_matrix = np.array([[np.cos(theta_l), -np.sin(theta_l)],
                           [np.sin(theta_l), np.cos(theta_l)]])
    corners_global = (rot_matrix @ corners_local.T).T + trailer_center
    trailer_poly = Polygon(corners_global, facecolor='lightcoral', edgecolor='red', 
                          linewidth=2, alpha=alpha, zorder=2)
    ax.add_patch(trailer_poly)
    patches_list.append(trailer_poly)
    
    # Draw wheels
    wheel_radius = 0.15
    wheel_offset = 0.25
    # Truck front wheels
    for side in [-1, 1]:
        wheel_center = truck_front + side * wheel_offset * np.array([-np.sin(theta_t), np.cos(theta_t)])
        wheel = Circle(wheel_center, wheel_radius, facecolor='black', edgecolor='darkgray', 
                      linewidth=1, zorder=4, alpha=alpha)
        ax.add_patch(wheel)
        patches_list.append(wheel)
    
    # Truck rear wheels (at hitch)
    for side in [-1, 1]:
        wheel_center = truck_rear + side * wheel_offset * np.array([-np.sin(theta_t), np.cos(theta_t)])
        wheel = Circle(wheel_center, wheel_radius, facecolor='black', edgecolor='darkgray', 
                      linewidth=1, zorder=4, alpha=alpha)
        ax.add_patch(wheel)
        patches_list.append(wheel)
    
    # Trailer wheels
    for side in [-1, 1]:
        wheel_center = trailer_rear + side * wheel_offset * np.array([-np.sin(theta_l), np.cos(theta_l)])
        wheel = Circle(wheel_center, wheel_radius, facecolor='black', edgecolor='darkgray', 
                      linewidth=1, zorder=4, alpha=alpha)
        ax.add_patch(wheel)
        patches_list.append(wheel)
    
    # Draw hitch point
    hitch_circle = Circle((x, y), 0.1, facecolor='black', edgecolor='white', 
                         linewidth=1, zorder=5, alpha=alpha)
    ax.add_patch(hitch_circle)
    patches_list.append(hitch_circle)
    
    # Draw direction arrows using FancyArrowPatch for proper removal
    arrow_length = 0.5
    # Truck direction
    arrow_t = FancyArrowPatch((x, y), 
                              (x + arrow_length*np.cos(theta_t), y + arrow_length*np.sin(theta_t)),
                              arrowstyle='->', mutation_scale=15, linewidth=2,
                              color='blue', alpha=alpha, zorder=6)
    ax.add_patch(arrow_t)
    patches_list.append(arrow_t)
    # Trailer direction
    arrow_l = FancyArrowPatch((x, y),
                              (x - arrow_length*np.cos(theta_l), y - arrow_length*np.sin(theta_l)),
                              arrowstyle='->', mutation_scale=15, linewidth=2,
                              color='red', alpha=alpha, zorder=6)
    ax.add_patch(arrow_l)
    patches_list.append(arrow_l)
    
    return patches_list

def animate_trajectory(q_hist, params, interval=80, max_frames=500):
    """
    Improved 2D animation with proper truck-trailer visualization.
    
    Parameters:
    -----------
    q_hist : array
        State history
    params : dict
        Parameters dictionary
    interval : int
        Animation interval in milliseconds
    max_frames : int
        Maximum number of frames to animate (downsample if needed)
    """
    q_hist = np.atleast_2d(q_hist)
    if q_hist.shape[0] == 6:
        q_hist = q_hist.T

    L = params["L"]
    d = params["d"]
    M_original = len(q_hist)
    
    # Downsample if too many frames to keep animation size manageable
    if M_original > max_frames:
        frame_step = M_original // max_frames
        q_hist = q_hist[::frame_step]
        M = len(q_hist)
        print(f"Downsampling animation: {M_original} frames -> {M} frames (step={frame_step})")
    else:
        frame_step = 1
        M = M_original

    fig, ax = plt.subplots(figsize=(12, 12))
    ax.set_aspect("equal")
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlabel("x (m)", fontsize=12)
    ax.set_ylabel("y (m)", fontsize=12)
    ax.set_title("Truck-Trailer MPC Navigation", fontsize=14, fontweight='bold')

    # Calculate proper limits including target
    all_x = np.concatenate([q_hist[:,0], [params["target"][0]]])
    all_y = np.concatenate([q_hist[:,1], [params["target"][1]]])
    margin = max(8.0, 0.25*max(np.ptp(all_x), np.ptp(all_y)))
    ax.set_xlim(all_x.min()-margin, all_x.max()+margin)
    ax.set_ylim(all_y.min()-margin, all_y.max()+margin)
    
    # Draw full path (hitch trajectory)
    path_line, = ax.plot(q_hist[:,0], q_hist[:,1], "k--", alpha=0.3, lw=1.5, label="Path (hitch)", zorder=1)
    
    # Draw initial position (beginning) as green star
    q0 = params["q0"]
    initial_x, initial_y = q0[0], q0[1]
    initial_marker = ax.plot(initial_x, initial_y, "g*", ms=20, mew=2, label="Start", zorder=10)[0]
    
    # Draw target1 if provided (for combined forward/reverse navigation)
    if "target1" in params:
        target1_x, target1_y = params["target1"][0], params["target1"][1]
        target1_marker = ax.plot(target1_x, target1_y, "mo", ms=18, mew=2, label="Target 1", zorder=10)[0]
    
    # Draw target with orientation if available
    target_x, target_y = params["target"][0], params["target"][1]
    target_marker = ax.plot(target_x, target_y, "r*", ms=20, mew=2, label="Target", zorder=10)[0]
    
    # Draw target orientation if provided
    if len(params["target"]) >= 4:
        target_theta_t = params["target"][2]
        target_theta_l = params["target"][3]
        # Draw target truck orientation
        ax.arrow(target_x, target_y, 1.5*np.cos(target_theta_t), 1.5*np.sin(target_theta_t),
                head_width=0.3, head_length=0.2, fc='green', ec='green', alpha=0.5, linestyle='--', zorder=9)
        # Draw target trailer orientation
        ax.arrow(target_x, target_y, -1.5*np.cos(target_theta_l), -1.5*np.sin(target_theta_l),
                head_width=0.3, head_length=0.2, fc='orange', ec='orange', alpha=0.5, linestyle='--', zorder=9)
    
    # Initialize patches list for animation
    patches_list = []
    
    title_text = ax.text(0.5, 0.98, "", ha="center", va="top", transform=ax.transAxes, 
                        fontsize=12, fontweight='bold', 
                        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    info_text = ax.text(0.02, 0.98, "", ha="left", va="top", transform=ax.transAxes,
                       fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

    def init():
        title_text.set_text("")
        info_text.set_text("")
        return [title_text, info_text]

    def update(frame):
        # Clear previous frame's patches
        for patch in patches_list:
            try:
                patch.remove()
            except:
                pass
        patches_list.clear()
        
        x, y, theta_t, theta_l = q_hist[frame, :4]
        v = q_hist[frame, 4] if q_hist.shape[1] > 4 else 0.0
        phi = q_hist[frame, 5] if q_hist.shape[1] > 5 else 0.0
        
        # Draw current truck-trailer (patches_list will be populated)
        draw_truck_trailer_2d(ax, x, y, theta_t, theta_l, L, d, alpha=1.0, patches_list=patches_list)
        
        # Update text
        time_val = frame * frame_step * params["dt"]
        dist_to_target = np.linalg.norm([x - target_x, y - target_y])
        title_text.set_text(f"Step {frame*frame_step+1}/{M_original} | Time: {time_val:.1f}s")
        info_text.set_text(f"Position: ({x:.2f}, {y:.2f})\n"
                          f"Truck θ: {np.degrees(theta_t):.1f}°\n"
                          f"Trailer θ: {np.degrees(theta_l):.1f}°\n"
                          f"Distance to target: {dist_to_target:.2f} m\n"
                          f"Velocity: {v:.2f} m/s\n"
                          f"Steering: {np.degrees(phi):.1f}°")
        
        return [title_text, info_text]

    anim = FuncAnimation(fig, update, init_func=init, frames=M, interval=interval, blit=False, repeat=True)
    ax.legend(loc="upper right", fontsize=10)
    return anim


### Forward Point-to-Point Navigation

In [ ]:
params_forward_point = {
    "L": 2.0, "d": 5.0,
    "target": np.array([np.random.uniform(0,15), np.random.uniform(-15,15), 0.0, 0.0]),
    "q0": np.array([0.0,0.0,0.0,0.0,0.0,0.0]),
    "dt": 0.1, "N": 10,
    "v_min": 0.0, "v_max": 3.0, "phi_min": -1.5, "phi_max": 1.5,
    "a_min": -2.0, "a_max": 2.0, "phi_dot_min": -1.0, "phi_dot_max": 1.0,
    "max_jackknife_angle": np.pi/2,
    "w_control": 0.01,
    "max_steps": 200, "target_tol": 0.5,
    "solver_max_iter": 500, "solver_tol": 1e-6, "solver_print_level": 0,
    "a_init": 0.1, "phi_dot_init": 0.0
}

q_hist_fwd, u_hist_fwd = run_mpc_loop(params_forward_point)
plot_trajectory(q_hist_fwd, u_hist_fwd, params_forward_point)

anim_fwd = animate_trajectory(q_hist_fwd, params_forward_point, interval=80)
display(HTML(anim_fwd.to_jshtml()))

### Reverse Point-to-Point Navigation

In [ ]:
params_reverse_point = {
    # Physical parameters
    "L": 2.0,           # truck wheelbase (m)
    "d": 5.0,           # hitch-to-trailer distance (m)

    # Target state [x, y, theta_t, theta_l]
    "target": np.array([np.random.uniform(-100, -80), np.random.uniform(-15, 15), 0.0, 0.0]),

    # Initial state [x, y, theta_t, theta_l, v, phi]
    "q0": np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),

    # MPC parameters
    "dt": 0.1,          # time step (s)
    "N": 200,            # prediction horizon

    # State bounds
    "v_min": -6.0,      # reverse only
    "v_max": 0.0,
    "phi_min": -1.5,
    "phi_max": 1.5,

    # Control bounds (acceleration & steering rate)
    "a_min": -2.0,
    "a_max": 2.0,
    "phi_dot_min": -0.5,
    "phi_dot_max": 0.5,

    # Constraints
    "max_jackknife_angle": np.pi/2,

    # Objective weights
    "w_control": 0.01,

    # Simulation parameters
    "max_steps": 1000,
    "target_tol": 0.5,

    # Solver settings
    "solver_max_iter": 500,
    "solver_tol": 1e-6,
    "solver_print_level": 0,

    # Initial guess
    "a_init": -0.1,      # small reverse acceleration
    "phi_dot_init": 0.0, # initial steering rate
}

# Run MPC
q_hist_rev, u_hist_rev = run_mpc_loop(params_reverse_point)

# Visualize results
plot_trajectory(q_hist_rev, u_hist_rev, params_reverse_point)

# Animate
anim_rev = animate_trajectory(q_hist_rev, params_reverse_point, interval=80)

# Display animation
display(HTML(anim_rev.to_jshtml()))


### Combined Forward and Reverse Navigation


In [ ]:
# Combined Forward and Reverse Navigation
# Step 1: Forward to Target 1
# Step 2: Reverse to Target 2
#
# NOTE: If you see "⚠ Reverse phase not shown - insufficient trajectory data":
#   This means reverse navigation failed (solver found it infeasible) or 
#   only generated 1 point (the starting position). This can happen when:
#   - Target 2 is not reachable with reverse-only constraints
#   - The initial guess is poor
#   - The solver cannot find a feasible path

# Check if required functions are defined
required_functions = ['run_mpc_loop', 'truck_trailer_dynamics', 'plot_trajectory', 'animate_trajectory']
missing_functions = []
for func_name in required_functions:
    if func_name not in globals():
        missing_functions.append(func_name)

if missing_functions:
    print("=" * 60)
    print("ERROR: Required functions not defined!")
    print("=" * 60)
    print(f"Missing functions: {', '.join(missing_functions)}")
    print("\nPlease run the following cells in order:")
    print("  1. Cell 2: Imports")
    print("  2. Cell 4: truck_trailer_dynamics function")
    print("  3. Cell 7: build_pyomo_model function")
    print("  4. Cell 9: run_mpc_loop function")
    print("  5. Cell 11: plot_trajectory and animate_trajectory functions")
    print("=" * 60)
    raise NameError(f"The following functions are not defined: {', '.join(missing_functions)}. Please run the cells above first.")

# Initial position
start_pos = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

# Target 1: Forward destination
target1 = np.array([100.0, 10.0, 0.0, 0.0])

# Target 2: Reverse destination (from Target 1)
target2 = np.array([50.0, 50.0, 0.0, 0.0])

print("=" * 60)
print("COMBINED FORWARD AND REVERSE NAVIGATION")
print("=" * 60)
print(f"Start position: ({start_pos[0]:.1f}, {start_pos[1]:.1f})")
print(f"Target 1 (forward): ({target1[0]:.1f}, {target1[1]:.1f}) - 100m forward")
print(f"Target 2 (reverse): ({target2[0]:.1f}, {target2[1]:.1f}) - 100m from Target 1")
print("=" * 60)

# ============================================================================
# PHASE 1: Forward Navigation to Target 1
# ============================================================================
print("\n>>> PHASE 1: Forward Navigation to Target 1")
print("-" * 60)

params_forward = {
    "L": 2.0, "d": 5.0,
    "target": target1,
    "q0": start_pos.copy(),
    "dt": 0.1, "N": 10,
    "v_min": 0.0, "v_max": 3.0, 
    "phi_min": -1.5, "phi_max": 1.5,
    "a_min": -2.0, "a_max": 2.0, 
    "phi_dot_min": -1.0, "phi_dot_max": 1.0,
    "max_jackknife_angle": np.pi/2,
    "w_control": 0.01,
    "max_steps": 500, "target_tol": 0.5,
    "solver_max_iter": 500, "solver_tol": 1e-6, "solver_print_level": 0,
    "a_init": 0.1, "phi_dot_init": 0.0
}

q_hist_forward, u_hist_forward = run_mpc_loop(params_forward)

# Get final state from forward navigation (this becomes start for reverse)
final_state_forward = q_hist_forward[-1].copy()
print(f"\n✓ Reached Target 1 at position: ({final_state_forward[0]:.2f}, {final_state_forward[1]:.2f})")
print(f"  Final error: {np.linalg.norm(final_state_forward[:2] - target1[:2]):.3f} m")

# ============================================================================
# PHASE 2: Reverse Navigation from Target 1 to Target 2
# ============================================================================
print("\n>>> PHASE 2: Reverse Navigation to Target 2")
print("-" * 60)

# Prepare initial state for reverse: keep position and orientation, but reset velocity and steering
# This helps the solver by starting from a "rest" state
q0_reverse = final_state_forward.copy()
q0_reverse[4] = 0.0  # Reset velocity to 0
q0_reverse[5] = 0.0  # Reset steering angle to 0
print(f"Starting reverse from: position=({q0_reverse[0]:.2f}, {q0_reverse[1]:.2f}), "
      f"theta_t={np.degrees(q0_reverse[2]):.1f}°, theta_l={np.degrees(q0_reverse[3]):.1f}°")
print(f"Target 2: ({target2[0]:.2f}, {target2[1]:.2f})")
print(f"Distance to target: {np.linalg.norm(q0_reverse[:2] - target2[:2]):.2f} m")

# Use EXACT same parameters as working reverse navigation (from Cell 15)
params_reverse = {
    # Physical parameters
    "L": 2.0,
    "d": 5.0,
    
    # Target state [x, y, theta_t, theta_l]
    "target": target2,
    
    # Initial state [x, y, theta_t, theta_l, v, phi] - with reset v and phi
    "q0": q0_reverse,
    
    # MPC parameters
    "dt": 0.1,
    "N": 200,  # Same as working reverse
    
    # State bounds
    "v_min": -6.0,  # reverse only
    "v_max": 0.0,
    "phi_min": -1.5,
    "phi_max": 1.5,
    
    # Control bounds (acceleration & steering rate)
    "a_min": -2.0,
    "a_max": 2.0,
    "phi_dot_min": -0.5,  # Same as working reverse (more conservative)
    "phi_dot_max": 0.5,
    
    # Constraints
    "max_jackknife_angle": np.pi/2,
    
    # Objective weights
    "w_control": 0.01,
    
    # Simulation parameters
    "max_steps": 1000,  # Same as working reverse
    "target_tol": 0.5,  # Same as working reverse (tighter tolerance)
    
    # Solver settings
    "solver_max_iter": 500,  # Same as working reverse
    "solver_tol": 1e-6,  # Same as working reverse
    "solver_print_level": 0,
    
    # Initial guess
    "a_init": -0.1,  # Same as working reverse (smaller initial acceleration)
    "phi_dot_init": 0.0,
}

q_hist_reverse, u_hist_reverse = run_mpc_loop(params_reverse)

# Check if reverse navigation succeeded
if len(q_hist_reverse) > 0:
    final_state_reverse = q_hist_reverse[-1].copy()
    final_error = np.linalg.norm(final_state_reverse[:2] - target2[:2])
    print(f"\n✓ Reached Target 2 at position: ({final_state_reverse[0]:.2f}, {final_state_reverse[1]:.2f})")
    print(f"  Final error: {final_error:.3f} m")
    
    if final_error > params_reverse["target_tol"]:
        print(f"  ⚠ Warning: Did not reach Target 2 within tolerance ({params_reverse['target_tol']} m)")
else:
    print(f"\n✗ Reverse navigation failed - no trajectory generated")
    print(f"  Using forward final state as reverse starting point")
    # Use forward final state as reverse state if reverse failed
    q_hist_reverse = final_state_forward.reshape(1, -1)
    u_hist_reverse = np.array([]).reshape(0, 2)

# ============================================================================
# Combine trajectories for visualization
# ============================================================================
print("\n>>> Combining trajectories for visualization")
print("-" * 60)

# Ensure arrays are properly shaped and not empty
# Convert to numpy arrays if they're lists
q_hist_forward = np.array(q_hist_forward) if not isinstance(q_hist_forward, np.ndarray) else q_hist_forward
q_hist_reverse = np.array(q_hist_reverse) if not isinstance(q_hist_reverse, np.ndarray) else q_hist_reverse
u_hist_forward = np.array(u_hist_forward) if not isinstance(u_hist_forward, np.ndarray) else u_hist_forward
u_hist_reverse = np.array(u_hist_reverse) if not isinstance(u_hist_reverse, np.ndarray) else u_hist_reverse

# Ensure 2D arrays
q_hist_forward = np.atleast_2d(q_hist_forward)
q_hist_reverse = np.atleast_2d(q_hist_reverse)
u_hist_forward = np.atleast_2d(u_hist_forward)
u_hist_reverse = np.atleast_2d(u_hist_reverse)

# Check shapes
print(f"Forward state history shape: {q_hist_forward.shape}")
print(f"Reverse state history shape: {q_hist_reverse.shape}")
print(f"Forward control history shape: {u_hist_forward.shape}")
print(f"Reverse control history shape: {u_hist_reverse.shape}")

# Fix control history shapes - they should be (N, 2)
if u_hist_forward.size == 0:
    u_hist_forward = np.array([]).reshape(0, 2)
elif len(u_hist_forward.shape) == 1:
    # If 1D, reshape to (N, 2) if possible
    if len(u_hist_forward) % 2 == 0:
        u_hist_forward = u_hist_forward.reshape(-1, 2)
    else:
        u_hist_forward = np.array([]).reshape(0, 2)
elif u_hist_forward.shape[1] != 2:
    # If wrong number of columns, fix it
    if u_hist_forward.shape[0] == 2 and u_hist_forward.shape[1] != 2:
        u_hist_forward = u_hist_forward.T
    else:
        u_hist_forward = np.array([]).reshape(0, 2)

if u_hist_reverse.size == 0:
    u_hist_reverse = np.array([]).reshape(0, 2)
elif len(u_hist_reverse.shape) == 1:
    # If 1D, reshape to (N, 2) if possible
    if len(u_hist_reverse) % 2 == 0:
        u_hist_reverse = u_hist_reverse.reshape(-1, 2)
    else:
        u_hist_reverse = np.array([]).reshape(0, 2)
elif u_hist_reverse.shape[1] != 2:
    # If wrong number of columns, fix it
    if u_hist_reverse.shape[0] == 2 and u_hist_reverse.shape[1] != 2:
        u_hist_reverse = u_hist_reverse.T
    else:
        u_hist_reverse = np.array([]).reshape(0, 2)

# Ensure state histories have correct shape (N, 6)
if q_hist_forward.shape[0] == 6 and q_hist_forward.shape[1] != 6:
    q_hist_forward = q_hist_forward.T
if q_hist_reverse.shape[0] == 6 and q_hist_reverse.shape[1] != 6:
    q_hist_reverse = q_hist_reverse.T

# Combine state histories (skip first element of reverse to avoid duplicate)
print(f"\nBefore combination:")
print(f"  Forward history length: {len(q_hist_forward)}")
print(f"  Reverse history length: {len(q_hist_reverse)}")

if len(q_hist_reverse) > 1:
    # Check if reverse actually moved (not just stayed at same point)
    reverse_distance = np.linalg.norm(q_hist_reverse[-1, :2] - q_hist_reverse[0, :2])
    print(f"  Reverse phase distance: {reverse_distance:.2f} m")
    
    if reverse_distance > 0.1:  # Only combine if reverse actually moved
        q_hist_combined = np.vstack([q_hist_forward, q_hist_reverse[1:]])
        print(f"  ✓ Successfully combined trajectories")
    else:
        q_hist_combined = np.vstack([q_hist_forward, q_hist_reverse[1:]])
        print(f"  ⚠ Reverse phase didn't move much, but including it anyway")
else:
    q_hist_combined = q_hist_forward.copy()
    print("  ⚠ Warning: Reverse history is too short, using only forward trajectory")

# Combine control histories
if len(u_hist_forward) > 0 and len(u_hist_reverse) > 0:
    u_hist_combined = np.vstack([u_hist_forward, u_hist_reverse])
elif len(u_hist_forward) > 0:
    u_hist_combined = u_hist_forward.copy()
    print("Warning: Reverse control history is empty, using only forward controls")
elif len(u_hist_reverse) > 0:
    u_hist_combined = u_hist_reverse.copy()
    print("Warning: Forward control history is empty, using only reverse controls")
else:
    u_hist_combined = np.array([]).reshape(0, 2)
    print("Warning: Both control histories are empty")

print(f"\nTotal trajectory points: {len(q_hist_combined)}")
print(f"Forward phase: {len(q_hist_forward)} steps")
print(f"Reverse phase: {len(q_hist_reverse)} steps")
if len(q_hist_combined) > 1:
    print(f"Total distance traveled: {np.sum(np.linalg.norm(np.diff(q_hist_combined[:, :2], axis=0), axis=1)):.2f} m")
else:
    print("Total distance traveled: 0.00 m (insufficient trajectory data)")

# ============================================================================
# Visualization
# ============================================================================
print("\n>>> Creating visualization")
print("-" * 60)

# Create custom params for combined visualization
params_combined = params_forward.copy()
params_combined["target"] = target2  # Show final target
params_combined["q0"] = start_pos  # Show starting position
params_combined["target1"] = target1  # Store target1 for visualization
params_combined["target2"] = target2  # Store target2 separately for visualization

# Plot combined trajectory
plot_trajectory(q_hist_combined, u_hist_combined, params_combined)

# Also create a custom plot showing both phases separately with time information
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# ===== LEFT PLOT: 2D Trajectory =====
# Plot forward phase in blue
forward_time = np.arange(len(q_hist_forward)) * params_forward["dt"]
if len(q_hist_forward) > 1:
    forward_end_idx = len(q_hist_forward)
    line1 = ax1.plot(q_hist_forward[:, 0], q_hist_forward[:, 1], "b-", linewidth=2.5, 
                     label=f"Forward Phase ({len(q_hist_forward)-1} steps, {forward_time[-1]:.1f}s)", 
                     alpha=0.8, zorder=2)

# Plot reverse phase in red (if it exists and has more than 1 point)
if len(q_hist_reverse) > 1:
    reverse_start_idx = len(q_hist_forward)
    reverse_time = np.arange(len(q_hist_reverse)) * params_reverse["dt"]
    # Plot reverse phase separately
    line2 = ax1.plot(q_hist_reverse[:, 0], q_hist_reverse[:, 1], "r--", linewidth=2.5, 
                     label=f"Reverse Phase ({len(q_hist_reverse)-1} steps, {reverse_time[-1]:.1f}s)", 
                     alpha=0.8, zorder=2)
    # Mark transition point
    transition_point = q_hist_reverse[0]
    ax1.plot(transition_point[0], transition_point[1], "ko", ms=8, mew=1.5, 
             label="Transition", zorder=9, markerfacecolor='yellow', markeredgecolor='black')
else:
    print("⚠ Reverse phase not shown - insufficient trajectory data")
    print("   This means reverse navigation failed or only generated 1 point (starting position)")

# Mark waypoints with smaller markers
ax1.plot(start_pos[0], start_pos[1], "g*", ms=12, mew=1.5, label="Start", zorder=10)
ax1.plot(target1[0], target1[1], "mo", ms=10, mew=1.5, label="Target 1", zorder=10)
ax1.plot(target2[0], target2[1], "r*", ms=12, mew=1.5, label="Target 2", zorder=10)

ax1.set_xlabel("x (m)", fontsize=11)
ax1.set_ylabel("y (m)", fontsize=11)
ax1.set_title("2D Trajectory", fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=9, loc='best', framealpha=0.9, markerscale=0.8)
ax1.set_aspect('equal')

# ===== RIGHT PLOT: Position vs Time =====
if len(q_hist_forward) > 1:
    ax2.plot(forward_time, q_hist_forward[:, 0], "b-", linewidth=2, label="x (Forward)", alpha=0.8)
    ax2.plot(forward_time, q_hist_forward[:, 1], "b--", linewidth=2, label="y (Forward)", alpha=0.8)

if len(q_hist_reverse) > 1:
    # Continue time from forward phase
    reverse_time_full = forward_time[-1] + params_reverse["dt"] + np.arange(len(q_hist_reverse)) * params_reverse["dt"]
    ax2.plot(reverse_time_full, q_hist_reverse[:, 0], "r-", linewidth=2, label="x (Reverse)", alpha=0.8)
    ax2.plot(reverse_time_full, q_hist_reverse[:, 1], "r--", linewidth=2, label="y (Reverse)", alpha=0.8)
    
    # Mark transition time
    ax2.axvline(x=forward_time[-1], color='orange', linestyle=':', linewidth=1.5, 
                label=f"Transition at {forward_time[-1]:.1f}s", alpha=0.7)

# Mark target positions
ax2.axhline(y=target1[0], color='m', linestyle=':', linewidth=1, alpha=0.5, label="Target 1 x")
ax2.axhline(y=target1[1], color='m', linestyle=':', linewidth=1, alpha=0.5, label="Target 1 y")
ax2.axhline(y=target2[0], color='r', linestyle=':', linewidth=1, alpha=0.5, label="Target 2 x")
ax2.axhline(y=target2[1], color='r', linestyle=':', linewidth=1, alpha=0.5, label="Target 2 y")

ax2.set_xlabel("Time (s)", fontsize=11)
ax2.set_ylabel("Position (m)", fontsize=11)
ax2.set_title("Position vs Time", fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=9, loc='best', framealpha=0.9, ncol=2)

plt.tight_layout()
plt.show()

# Animate combined trajectory (show all steps, no downsampling)
print("\nGenerating animation...")
anim_combined = animate_trajectory(q_hist_combined, params_combined, interval=80, max_frames=None)

# Display animation
display(HTML(anim_combined.to_jshtml()))

print("\n" + "=" * 60)
print("NAVIGATION COMPLETE!")
print("=" * 60)
